In [169]:
import numpy as np
from numpy import linalg as LA
from scipy.io import loadmat
import scipy
from mpl_toolkits import mplot3d
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib.pyplot import cm
import matplotlib as mpl
import cv2
import computer_vision as cv
from tqdm import trange
import time
from get_dataset_info import *
from scipy.spatial.transform import Rotation
# import cyvlfeat as vl

# %load_ext snakeviz
# %matplotlib inline
%matplotlib qt
%config InlineBackend.figure_format = 'retina'
from matplotlib import rc
rc('font', **{'family': 'serif', 'serif': ['Computer Modern']})
rc('text', usetex=True)

In [170]:
def plot_3D_points(X):
    
    fig = plt.figure()
    ax = plt.axes(projection='3d')
    ax.plot(X[0], X[1], X[2], '.', ms=1, color='magenta', label='X')
    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    # ax.set_box_aspect([1, 1, 1]) 
    ax.set_aspect('equal')
    # ax.view_init(elev=-50, azim=-104, roll=20)
    ax.legend(loc="lower right")
    fig.tight_layout()
    plt.show()

In [171]:
def compute_E_validity(E):
    rank = LA.matrix_rank(E)
    valid = True if rank == 2 else False
    return valid

def compute_E_inliers(E, x1_norm, x2_norm, err_threshold):
    
    distance1_arr, distance2_arr = cv.compute_epipolar_errors(E, x1_norm, x2_norm)
    inliers = ((distance1_arr**2 + distance2_arr**2) / 2) < err_threshold**2
    n_inliers = np.sum(inliers)
    epsilon_E = n_inliers / x1_norm.shape[1]

    return epsilon_E, inliers

def verbose_E_robust(t, T_E, T_H, epsilon_E, epsilon_H, inliers, method):
    print('Iteration:', t, 'T_E:', T_E, 'T_H:', T_H, 'epsilon_E:', np.round(epsilon_E, 2), 'epsilon_H:', np.round(epsilon_H, 2), 'No. inliers:', np.sum(inliers), 'From:', method)


def estimate_E_robust(K, x1_norm, x2_norm, min_its, max_its, scale_its, alpha, err_threshold_px, essential_matrix=True, homography=True, verbose=False):
    
    err_threshold = err_threshold_px / K[0,0]
    best_E = None
    best_inliers = None
    n_points = x1_norm.shape[1]
    n_E_samples = 8
    n_H_samples = 4
    best_epsilon_E = 0
    best_epsilon_H = 0
    T_E = max_its
    T_H = max_its

    t = 0
    while t < T_E and t < T_H:
        t += 1

        if essential_matrix:
            rand_mask = np.random.choice(n_points, n_E_samples, replace=False)
            E = cv.estimate_E_DLT(x1_norm[:,rand_mask], x2_norm[:,rand_mask], enforce=True, verbose=False)
            E_valid = compute_E_validity(E)

            if E_valid:
                epsilon_E, inliers = compute_E_inliers(E, x1_norm, x2_norm, err_threshold)
                    
                if epsilon_E > best_epsilon_E:
                    best_E = np.copy(E)
                    best_inliers = np.copy(inliers)
                    best_epsilon_E = epsilon_E
                    T_E = cv.compute_ransac_iterations(alpha, best_epsilon_E, n_E_samples, min_its, max_its, scale_its)

                    if verbose:
                        verbose_E_robust(t, T_E, T_H, best_epsilon_E, best_epsilon_H, best_inliers, method='E 8-point alg.')
        
        if homography:
            rand_mask = np.random.choice(n_points, n_H_samples, replace=False)
            H = cv.estimate_H_DLT(x1_norm[:,rand_mask], x2_norm[:,rand_mask], verbose=False)
            x2_norm_proj = cv.dehomogenize(H @ x1_norm)
            distance_arr = cv.compute_point_point_distance(x2_norm_proj, x2_norm)
            inliers = distance_arr < err_threshold
            n_inliers = np.sum(inliers)
            epsilon_H = n_inliers / n_points

            if epsilon_H > best_epsilon_H:
                
                # num, Rs, Ts, Ns = cv2.decomposeHomographyMat(H, np.eye(3))
                R1, T1, R2, T2 = cv.homography_to_RT(H, x1_norm, x2_norm)
                E1 = cv.compute_E_from_R_and_T(R1, T1)
                E2 = cv.compute_E_from_R_and_T(R2, T2)

                E1_valid = compute_E_validity(E1)
                E2_valid = compute_E_validity(E2)

                if E1_valid:
                    epsilon_E, inliers = compute_E_inliers(E1, x1_norm, x2_norm, err_threshold)
                        
                    if epsilon_E > best_epsilon_E:
                        best_E = np.copy(E1)
                        best_inliers = np.copy(inliers)
                        best_epsilon_E = epsilon_E
                        best_epsilon_H = epsilon_H
                        T_E = cv.compute_ransac_iterations(alpha, best_epsilon_E, n_E_samples, min_its, max_its, scale_its)
                        T_H = cv.compute_ransac_iterations(alpha, best_epsilon_H, n_H_samples, min_its, max_its, scale_its)

                        if verbose:
                            verbose_E_robust(t, T_E, T_H, best_epsilon_E, best_epsilon_H, best_inliers, method='H 4-point alg.')

                if E2_valid:
                    epsilon_E, inliers = compute_E_inliers(E2, x1_norm, x2_norm, err_threshold)
                        
                    if epsilon_E > best_epsilon_E:
                        best_E = np.copy(E2)
                        best_inliers = np.copy(inliers)
                        best_epsilon_E = epsilon_E
                        best_epsilon_H = epsilon_H
                        T_E = cv.compute_ransac_iterations(alpha, best_epsilon_E, n_E_samples, min_its, max_its, scale_its)
                        T_H = cv.compute_ransac_iterations(alpha, best_epsilon_H, n_H_samples, min_its, max_its, scale_its)
                        
                        if verbose:
                            verbose_E_robust(t, T_E, T_H, best_epsilon_E, best_epsilon_H, best_inliers, method='H 4-point alg.')
        
    print('Bailout at iteration:', t)
    return best_E, best_inliers

In [5]:
def compute_sift_points(img1, img2, marg, flann=False, verbose=False):
    img1 = cv2.cvtColor(img1, cv2.COLOR_BGR2GRAY)
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    sift = cv2.SIFT_create()
    kp1, des1 = sift.detectAndCompute(img1, None)
    kp2, des2 = sift.detectAndCompute(img2, None)

    if flann:
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks=50)   # or pass empty dictionary

        flann = cv2.FlannBasedMatcher(index_params, search_params)
        matches = flann.knnMatch(des1, des2, k=2)
    else:
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(des1, des2, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < marg*n.distance:
            good_matches.append([m])

    x1 = np.stack([kp1[match[0].queryIdx].pt for match in good_matches],1)
    x2 = np.stack([kp2[match[0].trainIdx].pt for match in good_matches],1)
    x1 = cv.homogenize(x1, multi=True)
    x2 = cv.homogenize(x2, multi=True)

    des1 = np.stack([des1[match[0].queryIdx] for match in good_matches],0)
    des2 = np.stack([des2[match[0].trainIdx] for match in good_matches],0)

    if verbose:
        print('Number of matches:', np.size(matches,0))
        print('Number of good matches:', np.size(x1,1))

    return x1, x2, des1, des2

def compute_sift_points_TR(x1, des1, img2, marg, flann=False, verbose=False):
    img2 = cv2.cvtColor(img2, cv2.COLOR_BGR2GRAY)

    # nfeatures=0
    # sigma=1.6
    # contrastThreshold=0.04
    # edgeThreshold=10

    # nfeatures=nfeatures,
    # sigma=2/sigma,  
    # contrastThreshold=2/contrastThreshold,
    # edgeThreshold=2/edgeThreshold,

    sift = cv2.SIFT_create()
    kp2, des2 = sift.detectAndCompute(img2, None)

    if flann:
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
        search_params = dict(checks=50)   # or pass empty dictionary

        flann = cv2.FlannBasedMatcher(index_params, search_params)
        matches = flann.knnMatch(des1, des2, k=2)
    else:
        bf = cv2.BFMatcher()
        matches = bf.knnMatch(des1, des2, k=2)

    good_matches = []
    for m, n in matches:
        if m.distance < marg*n.distance:
            good_matches.append([m])

    x_idx = np.array([match[0].queryIdx for match in good_matches])
    x1 = np.stack([x1[:,match[0].queryIdx] for match in good_matches],1)
    x2 = np.stack([kp2[match[0].trainIdx].pt for match in good_matches],1)
    x2 = cv.homogenize(x2, multi=True)

    if verbose:
        print('Number of matches:', np.size(matches,0))
        print('Number of good matches:', np.size(x1,1))

    return x1, x2, x_idx

In [215]:
def plot_cameras_and_axes(ax, C_list, axis_list, s, valid_idx, origin):

    col = cm.rainbow(np.linspace(0, 1, np.size(C_list,1)))

    for i in range(np.size(C_list,1)):

        C = C_list[:,i]
        axis = axis_list[:,i]

        if valid_idx[i] == origin:
            canonical = ' (canonical)'
        else:
            canonical = ''
        ax.plot(C[0], C[1], C[2], 'o', color=col[i],  label='Camera {}'.format(valid_idx[i]+1)+canonical, alpha=0.7)

        x_axis = C[0] + s*axis[0]
        y_axis = C[1] + s*axis[1]
        z_axis = C[2] + s*axis[2]

        ax.plot([x_axis, C[0]], [y_axis, C[1]], [z_axis, C[2]], '-', color=col[i], lw=3, alpha=0.7)

def plot_cameras_and_3D_points(X, C_arr, axis_arr, s, valid_idx, origin, path, save=False):
    
    fig = plt.figure(figsize=(8,6))
    ax = plt.axes(projection='3d')

    ax.plot(X[0], X[1], X[2], '.', ms=0.2, color='magenta', label='3D points')
    plot_cameras_and_axes(ax, C_arr, axis_arr, s, valid_idx, origin)

    ax.set_xlabel('$x$')
    ax.set_ylabel('$y$')
    ax.set_zlabel('$z$')
    ax.set_aspect('equal')
    # ax.view_init(elev=-50, azim=-104, roll=45)
    plt.legend(loc="lower right")
    if save:
        fig.savefig(path, dpi=300)
    plt.show()

In [7]:
def compute_feasible_points(P1, P2, X, percentile):
    
    x1 = P1 @ X
    x2 = P2 @ X
    x1_filter = x1[-1,:] > 0
    x2_filter = x2[-1,:] > 0

    X_bar = np.mean(X, axis=1)
    X_norm = LA.norm(X - X_bar[:,None], axis=0)
    norm_percentile = np.percentile(X_norm, percentile)
    outlier_filter = X_norm < norm_percentile

    feasible_pts = x1_filter * x2_filter * outlier_filter
    return feasible_pts

In [8]:
def compute_absolute_rotations(rel_rots, origin_idx, verbose=False):
    
    abs_rots = [rel_rots[0]]
    for i in range(len(rel_rots)-1):

        Ri = abs_rots[i]
        R2 = rel_rots[i+1]
        if LA.det(R2) < 0:
            print('WARNING: det(R{}) < 0, not a rotation!'.format(i), LA.det(R2))
            R2 = -R2
        U, _, VT = LA.svd(R2, full_matrices=False)
        R2 = U @ VT
        Rj = R2 @ Ri
        abs_rots.append(Rj)

    R0 = abs_rots[origin_idx]    
    for i in range(len(abs_rots)):

        Ri = abs_rots[i]
        Ri = LA.inv(R0) @ Ri
        abs_rots[i] = Ri

        if verbose:
            print('det(R{}):'.format(i), LA.det(Ri))
        
    return np.array(abs_rots)

In [9]:
def compute_rots_trans_RA(rel_cameras):

    P1 = rel_cameras[0]
    rots = [P1[:,:-1]]
    trans = [P1[:,-1]]

    for i in range(rel_cameras.shape[0]-1):
        P2 = rel_cameras[i+1]
        P2 /= P2[-1,2]
        R2 = P2[:,:-1]
        T2 = P2[:,-1]

        Ri = rots[i]
        Ti = trans[i]

        Rj = R2 @ Ri
        Tj = T2 + (Rj @ Ri.T @ Ti)

        rots.append(Rj)
        trans.append(Tj)

    rots = np.array(rots)
    trans = np.array(trans)

    return rots, trans

In [10]:
data_set = 1
K, img_names, init_pair, pixel_threshold = get_dataset_info(data_set)
# pixel_threshold *= 1
K_inv = LA.inv(K)
imgs = cv.load_image(img_names, multi=True)
n_imgs = imgs.shape[0]
n_camera_pairs = n_imgs - 1
img1_init = imgs[init_pair[0]]
img2_init = imgs[init_pair[1]]

In [175]:
print('\n\n\n### Computing rotation averaging ###\n')

marg = 0.75
min_its = 0
max_its = 10000
scale_its = 4
alpha = 0.99
P1 = cv.get_canonical_camera()
rel_cameras = [P1]

sift = True
ransac = True
extract = True
plot = True
rot = True

for i in range(n_camera_pairs):    
    print('\nCamera pair:', i+1, '/', n_camera_pairs)

    if sift:
        img1 = imgs[i]
        img2 = imgs[i+1]
        x1, x2, _, _ = compute_sift_points(img1, img2, marg, flann=True, verbose=True)
        np.save('data/dataset_{}_RA_x1_{}.npy'.format(data_set, i), x1)
        np.save('data/dataset_{}_RA_x2_{}.npy'.format(data_set, i), x2)   

    x1 = np.load('data/dataset_{}_RA_x1_{}.npy'.format(data_set, i))
    x2 = np.load('data/dataset_{}_RA_x2_{}.npy'.format(data_set, i))
    x1_norm = cv.dehomogenize(K_inv @ x1)
    x2_norm = cv.dehomogenize(K_inv @ x2)

    if ransac:
        E, inliers = estimate_E_robust(K, x1_norm, x2_norm, min_its, max_its, scale_its, alpha, pixel_threshold, essential_matrix=True, homography=True, verbose=True)
        np.save('data/dataset_{}_RA_E_{}.npy'.format(data_set, i), E)
        np.save('data/dataset_{}_RA_E_inliers_{}.npy'.format(data_set, i), inliers)

    if extract:
        E = np.load('data/dataset_{}_RA_E_{}.npy'.format(data_set, i))
        inliers = np.load('data/dataset_{}_RA_E_inliers_{}.npy'.format(data_set, i))

        x1_norm_inliers = x1_norm[:,inliers]
        x2_norm_inliers = x2_norm[:,inliers]

        P2_arr = cv.extract_P_from_E(E)
        X_arr = cv.compute_triangulated_X_from_extracted_P2_solutions(P1, P2_arr, x1_norm_inliers, x2_norm_inliers)
        P2, X = cv.extract_valid_camera_and_points(P1, P2_arr, X_arr, verbose=True)
        rel_cameras.append(P2)

        if plot:
            percentile = 90
            feasable_pts = compute_feasible_points(P1, P2, X, percentile)

            P_arr = np.array([P1, P2])
            C_arr, axis_arr = cv.compute_camera_center_and_normalized_principal_axis(P_arr, multi=True)
            plot_cameras_and_3D_points(X[:,feasable_pts], C_arr, axis_arr, s=1, path=None, save=False)

if rot:
    rel_cameras = np.array(rel_cameras)
    rel_rots = rel_cameras[:,:,:-1]
    abs_rots = compute_absolute_rotations(rel_rots, init_pair[0], verbose=True)
    rots_RA, trans_RA = compute_rots_trans_RA(rel_cameras)

    np.save('data/dataset_{}_RA_abs_rots.npy'.format(data_set), abs_rots)
    np.save('data/dataset_{}_RA_rots.npy'.format(data_set), rots_RA)
    np.save('data/dataset_{}_RA_trans.npy'.format(data_set), trans_RA)




### Computing rotation averaging ###


Camera pair: 1 / 8


Number of matches: 31591
Number of good matches: 13102
Iteration: 1 T_E: 10000 T_H: 10000 epsilon_E: 0.03 epsilon_H: 0 No. inliers: 397 From: E 8-point alg.
Iteration: 2 T_E: 10000 T_H: 10000 epsilon_E: 0.06 epsilon_H: 0 No. inliers: 833 From: E 8-point alg.
Iteration: 19 T_E: 10000 T_H: 10000 epsilon_E: 0.12 epsilon_H: 0 No. inliers: 1567 From: E 8-point alg.
Iteration: 70 T_E: 10000 T_H: 10000 epsilon_E: 0.14 epsilon_H: 0.0 No. inliers: 1884 From: H 4-point alg.
Iteration: 135 T_E: 10000 T_H: 10000 epsilon_E: 0.21 epsilon_H: 0.0 No. inliers: 2696 From: E 8-point alg.
Iteration: 157 T_E: 10000 T_H: 10000 epsilon_E: 0.42 epsilon_H: 0.0 No. inliers: 5447 From: E 8-point alg.
Iteration: 894 T_E: 10000 T_H: 10000 epsilon_E: 0.42 epsilon_H: 0.0 No. inliers: 5530 From: E 8-point alg.
Iteration: 1067 T_E: 3224.0 T_H: 10000 epsilon_E: 0.52 epsilon_H: 0.0 No. inliers: 6868 From: E 8-point alg.
Iteration: 1317 T_E: 1928.0 T_H: 10000 epsilon_E: 0.56 epsilon_H: 0.0 No. inliers: 7322 From: E 8-poi

In [11]:
print('\n\n\n### Computing initial 3D-points ###\n')

min_its = 10000
max_its = 20000
scale_its = 3
alpha = 0.99

sift = False
ransac = False
extract = True

if sift:
    marg = 0.75
    x1_init, x2_init, des1_init, des2_init = compute_sift_points(imgs[init_pair[0]], imgs[init_pair[1]], marg, flann=True, verbose=True)
    np.save('data/dataset_{}_TR_x1_{}.npy'.format(data_set, init_pair[1]), x1_init)
    np.save('data/dataset_{}_TR_x2_{}.npy'.format(data_set, init_pair[1]), x2_init)
    np.save('data/dataset_{}_TR_des1_init.npy'.format(data_set), des1_init)
    np.save('data/dataset_{}_TR_des2_init.npy'.format(data_set), des2_init)

x1_init = np.load('data/dataset_{}_TR_x1_{}.npy'.format(data_set, init_pair[1]))
x2_init = np.load('data/dataset_{}_TR_x2_{}.npy'.format(data_set, init_pair[1]))
x1_init_norm = cv.dehomogenize(K_inv @ x1_init)
x2_init_norm = cv.dehomogenize(K_inv @ x2_init)
des1_init = np.load('data/dataset_{}_TR_des1_init.npy'.format(data_set))
des2_init = np.load('data/dataset_{}_TR_des2_init.npy'.format(data_set))

if ransac:
    E, inliers = estimate_E_robust(K, x1_init_norm, x2_init_norm, min_its, max_its, scale_its, alpha, 3*pixel_threshold, essential_matrix=True, homography=True, verbose=True)
    np.save('data/dataset_{}_TR_E.npy'.format(data_set), E)
    np.save('data/dataset_{}_TR_E_inliers.npy'.format(data_set), inliers)

if extract:
    E = np.load('data/dataset_{}_TR_E.npy'.format(data_set))
    inliers = np.load('data/dataset_{}_TR_E_inliers.npy'.format(data_set))
    x1_init_norm_inliers = x1_init_norm[:,inliers]
    x2_init_norm_inliers = x2_init_norm[:,inliers]
    des1_init_inliers = des1_init[inliers]
    des2_init_inliers = des2_init[inliers]

    P1 = cv.get_canonical_camera()
    P2_arr = cv.extract_P_from_E(E)
    X_arr = cv.compute_triangulated_X_from_extracted_P2_solutions(P1, P2_arr, x1_init_norm_inliers, x2_init_norm_inliers)
    P2, X_init_inliers = cv.extract_valid_camera_and_points(P1, P2_arr, X_arr, verbose=True)

    # P_arr = np.array([P1, P2])
    # C_arr, axis_arr = cv.compute_camera_center_and_normalized_principal_axis(P_arr, multi=True)
    # plot_cameras_and_3D_points(X_init[:,inliers], C_arr, axis_arr, s=1, path=None, save=False)

    # R_init = rots[init_pair[0]]
    # X_init = LA.inv(R_init) @ X_init[:-1,:]

    percentile = 90
    feasible_pts = compute_feasible_points(P1, P2, X_init_inliers, percentile)

    x1_init_norm_feasible_inliers = x1_init_norm_inliers[:,feasible_pts]
    x2_init_norm_feasible_inliers = x2_init_norm_inliers[:,feasible_pts]
    des1_init_feasible_inliers = des1_init_inliers[feasible_pts]
    des2_init_feasible_inliers = des2_init_inliers[feasible_pts]
    X_init_feasible_inliers = X_init_inliers[:,feasible_pts]
    
    X_init_idx = np.ones(X_init_feasible_inliers.shape[1], dtype=bool)
    np.save('data/dataset_{}_TR_x2_norm_{}.npy'.format(data_set, init_pair[0]), x1_init_norm_feasible_inliers)
    np.save('data/dataset_{}_TR_x1_norm_{}.npy'.format(data_set, init_pair[1]), x1_init_norm_feasible_inliers)
    np.save('data/dataset_{}_TR_x2_norm_{}.npy'.format(data_set, init_pair[1]), x2_init_norm_feasible_inliers)
    np.save('data/dataset_{}_TR_X_idx_{}.npy'.format(data_set, init_pair[0]), X_init_idx)
    np.save('data/dataset_{}_TR_X_idx_{}.npy'.format(data_set, init_pair[1]), X_init_idx)

    print(x1_init_norm_feasible_inliers.shape, x2_init_norm_feasible_inliers.shape, des1_init_feasible_inliers.shape, des2_init_feasible_inliers.shape, X_init_feasible_inliers.shape, X_init_idx.shape)

    plot_3D_points(X_init_feasible_inliers)




### Computing initial 3D-points ###

No. valid coords for each camera pair: [2892    0 1446 1446]
Argmax(P2_arr): 0
(3, 1301) (3, 1301) (1301, 128) (1301, 128) (4, 1301) (1301,)


In [51]:
def estimate_T_robust(K, R, X, x_norm, min_its, max_its, scale_its, alpha, err_threshold_px, DLT1=False, verbose=False):
    
    err_threshold = err_threshold_px / K[0,0]
    best_T = np.full(3, np.nan)
    best_inliers = np.zeros(x_norm.shape[1], dtype=bool)
    best_epsilon = 0
    n_points = x_norm.shape[1]
    n_samples = 2
    ransac_its = max_its

    t = 0
    while t < ransac_its:
        t += 1

        rand_mask = np.random.choice(n_points, n_samples, replace=False)
        if DLT1:
            T = cv.estimate_T_DLT_1(x_norm[:,rand_mask], verbose=False)
        else:
            T = cv.estimate_T_DLT_2(R, x_norm[:,rand_mask], verbose=False)

        x_norm_proj = cv.dehomogenize(R @ X + T[:,None])
        distance_arr = cv.compute_point_point_distance(x_norm_proj, x_norm)
        inliers = distance_arr < err_threshold
        n_inliers = np.sum(inliers)
        epsilon = n_inliers / n_points

        if epsilon > best_epsilon:
            best_T = np.copy(T)
            best_inliers = np.copy(inliers)
            best_epsilon = epsilon
            ransac_its = cv.compute_ransac_iterations(alpha, best_epsilon, n_samples, min_its, max_its, scale_its)
            if verbose:
                print('Iteration:', t, 'T:', ransac_its, 'epsilon:', np.round(best_epsilon, 2), 'No. inliers:', np.sum(inliers))
    
    print('Bailout at iteration:', t)
    return best_T, best_inliers

In [55]:
print('\n\n\n### Computing translation registrations ###\n')

marg = 0.75
min_its = 0
max_its = 5000
scale_its = 1
alpha = 0.99
trans = []

des_X = des1_init_feasible_inliers.copy()
x_norm_feasible_inliers = x1_init_norm_feasible_inliers.copy()
abs_rots = np.load('data/dataset_{}_RA_abs_rots.npy'.format(data_set))
keep_rots = np.ones(abs_rots.shape[0], dtype=bool)

sift = False
plot = False
ransac = True

for i in range(n_imgs):
    print('\nImage:', i+1, '/', n_imgs)

    if i != init_pair[0]:

        if sift and i != init_pair[1]:
            img2 = imgs[i]
            
            # if i < int((init_pair[0]+init_pair[1])/2):
            #     des_X = des1_init_feasible_inliers.copy()
            #     x_norm_feasible_inliers = x1_init_norm_feasible_inliers.copy()
            # else:
            #     des_X = des2_init_feasible_inliers.copy()
            #     x_norm_feasible_inliers = x2_init_norm_feasible_inliers.copy()
            
            x1_norm, x2, X_idx = compute_sift_points_TR(x_norm_feasible_inliers, des_X, img2, marg, flann=True, verbose=True)
            x2_norm = cv.dehomogenize(K_inv @ x2)

            np.save('data/dataset_{}_TR_x1_norm_{}.npy'.format(data_set, i), x1_norm)
            np.save('data/dataset_{}_TR_x2_norm_{}.npy'.format(data_set, i), x2_norm)
            np.save('data/dataset_{}_TR_X_idx_{}.npy'.format(data_set, i), X_idx)

        x_norm = np.load('data/dataset_{}_TR_x2_norm_{}.npy'.format(data_set, i))
        X_idx = np.load('data/dataset_{}_TR_X_idx_{}.npy'.format(data_set, i))
        X = X_init_feasible_inliers[:,X_idx]        
        R = abs_rots[i]
        print(x_norm.shape, X.shape)

        if plot:
            plot_3D_points(X)

        if ransac:
            T, inliers = estimate_T_robust(K, R, X[:-1], x_norm, min_its, max_its, scale_its, alpha, 30*pixel_threshold, DLT1=False, verbose=True)
            # T = cv.estimate_T_DLT_2(R, x_norm[:,inliers], verbose=False)            
    else:
        x_norm = np.load('data/dataset_{}_TR_x2_norm_{}.npy'.format(data_set, i))
        X_idx = np.load('data/dataset_{}_TR_X_idx_{}.npy'.format(data_set, i))
        X = X_init_feasible_inliers[:,X_idx]        
        print(x_norm.shape, X.shape)

        if ransac:
            T = np.zeros(3)
            inliers = np.ones(x_norm.shape[1], dtype=bool)
    
    if ransac:
        trans.append(T)
        np.save('data/dataset_{}_TR_T_inliers_{}.npy'.format(data_set, i), inliers)
            
        if np.isnan(T[0]):
            keep_rots[i] = False

if ransac:
    trans = np.array(trans)
    np.save('data/dataset_{}_TR_trans.npy'.format(data_set), trans)
    np.save('data/dataset_{}_TR_keep_rots.npy'.format(data_set), keep_rots)




### Computing translation registrations ###


Image: 1 / 9
(3, 1301) (4, 1301)
[0. 0. 0.]

Image: 2 / 9
(3, 983) (4, 983)
Iteration: 4 T: 5000 epsilon: 0.0 No. inliers: 3
Iteration: 36 T: 5000 epsilon: 0.01 No. inliers: 9
Iteration: 74 T: 5000 epsilon: 0.02 No. inliers: 18
Iteration: 93 T: 5000 epsilon: 0.03 No. inliers: 25
Iteration: 265 T: 1321.0 epsilon: 0.06 No. inliers: 58
Iteration: 853 T: 1085.0 epsilon: 0.07 No. inliers: 64
Iteration: 944 T: 881.0 epsilon: 0.07 No. inliers: 71
Bailout at iteration: 944
[-0.22935842  0.20988982  0.63509132]

Image: 3 / 9
(3, 1021) (4, 1021)
Iteration: 10 T: 5000 epsilon: 0.01 No. inliers: 11
Iteration: 27 T: 5000 epsilon: 0.01 No. inliers: 14
Iteration: 46 T: 5000 epsilon: 0.02 No. inliers: 23
Iteration: 192 T: 5000 epsilon: 0.03 No. inliers: 27
Iteration: 222 T: 4406.0 epsilon: 0.03 No. inliers: 33
Iteration: 237 T: 1774.0 epsilon: 0.05 No. inliers: 52
Iteration: 245 T: 1170.0 epsilon: 0.06 No. inliers: 64
Iteration: 533 T: 808.0 epsilon: 0.

In [202]:
def compute_residual(params, x, X, R):
    T = params[0:3]
    x_proj = cv.dehomogenize(R @ X + T[:,None])
    residual = x_proj - x
    return residual.ravel()


print('\n\n\n### Refining translation vectors ###\n')

abs_rots = np.load('data/dataset_{}_RA_abs_rots.npy'.format(data_set))
trans = np.load('data/dataset_{}_TR_trans.npy'.format(data_set))
keep_rots = np.load('data/dataset_{}_TR_keep_rots.npy'.format(data_set))

abs_rots_opt = []
trans_opt = []

RA = False

if RA:
    trans = np.load('data/dataset_{}_RA_trans.npy'.format(data_set))
    keep_rots = np.ones(abs_rots.shape[0], dtype=bool)

for i in range(n_imgs):
    print('Translation:', i+1, '/', n_imgs)
    T = trans[i]
    
    if not np.isnan(T[0]):

        # if i != init_pair[0]:
    
        x_norm = np.load('data/dataset_{}_TR_x2_norm_{}.npy'.format(data_set, i))
        X_idx = np.load('data/dataset_{}_TR_X_idx_{}.npy'.format(data_set, i))
        if RA:
            inliers_T = np.ones(x_norm.shape[1], dtype=bool)
        else:
            inliers_T = np.load('data/dataset_{}_TR_T_inliers_{}.npy'.format(data_set, i))
        
        # inliers_T = np.ones(x_norm.shape[1], dtype=bool)
        
        x_norm = x_norm[:,inliers_T]
        X = X_init_feasible_inliers[:,X_idx][:,inliers_T]
        R = abs_rots[i]
        x0 = T.ravel()
        result = scipy.optimize.least_squares(compute_residual, x0, method='lm', args=(x_norm, X[:-1], R))
        T_opt = result.x

        trans_opt.append(T_opt)
        # else:
        #     trans_opt.append(trans[i])
    else:
        trans_opt.append(trans[i])
    
trans_opt = np.array(trans_opt)
np.save('data/dataset_{}_LM_trans_opt.npy'.format(data_set), trans_opt)




### Refining translation vectors ###

Translation: 1 / 9
Translation: 2 / 9
Translation: 3 / 9
Translation: 4 / 9
Translation: 5 / 9
Translation: 6 / 9
Translation: 7 / 9
Translation: 8 / 9
Translation: 9 / 9


In [184]:
def compute_reprojection_error_wrt_T(Pi, X, x):
    
    x_proj = cv.dehomogenize(Pi @ X)
    res = (x[:-1] - x_proj[:-1]).T.reshape(-1)
    reproj_err = LA.norm(res)**2
    
    return reproj_err, res

def compute_total_reprojection_error_wrt_T(P_arr, X_init, X_idx_arr, x_arr, inliers_arr, verbose=False):

    n_cameras = P_arr.shape[0]
    reproj_err_tot = []
    res_tot = []

    for i in range(n_cameras):

        Pi = P_arr[i]
        Ti = Pi[:,-1].copy()

        if not np.isnan(Ti[0]):

            inliers = inliers_arr[i]
            X_idx = X_idx_arr[i]

            X = X_init[:,X_idx][:,inliers]
            x = x_arr[i][:,inliers]

            reproj_err, res = compute_reprojection_error_wrt_T(Pi, X, x)
            reproj_err_tot.append(reproj_err)
            res_tot.append(res)
    
    res_tot = np.concatenate(res_tot, 0)

    if verbose:
        print('\nTotal reprojection error:', round(np.sum(reproj_err_tot), 2))
        print('Median reprojection error:', round(np.median(reproj_err_tot), 2))
        print('Avg. reprojection error:', round(np.mean(reproj_err_tot), 2))

    return reproj_err_tot, res_tot

def compute_jacobian_of_residual_wrt_T(Pi, Xj):
    jac1 = ((Pi[0,:] @ Xj) / (Pi[-1,:] @ Xj)**2) - (1 / (Pi[-1,:] @ Xj))
    jac2 = ((Pi[1,:] @ Xj) / (Pi[-1,:] @ Xj)**2) - (1 / (Pi[-1,:] @ Xj))
    jac = np.row_stack((jac1, jac2))
    return jac

def compute_jacobian_of_residual_wrt_T1(Pi, Xj):
    jac1 = -1 / (Pi[-1,:] @ Xj)
    jac2 = np.zeros(Xj.shape[1])
    jac = np.row_stack((jac1, jac2))
    return jac
 
def compute_jacobian_of_residual_wrt_T2(Pi, Xj):
    jac1 = np.zeros(Xj.shape[1])
    jac2 = -1 / (Pi[-1,:] @ Xj)
    jac = np.row_stack((jac1, jac2))
    return jac

def compute_jacobian_of_residual_wrt_T3(Pi, Xj):
    jac1 = (Pi[0,:] @ Xj) / (Pi[-1,:] @ Xj)**2
    jac2 = (Pi[1,:] @ Xj) / (Pi[-1,:] @ Xj)**2
    jac = np.row_stack((jac1, jac2))
    return jac

def linearize_reprojection_error_wrt_T(Pi, X, x):

    _, res = compute_reprojection_error_wrt_T(Pi, X, x)

    # jac = []
    # for j in range(X.shape[1]):
    #     jac_i = compute_jacobian_of_residual_wrt_T(Pi, X[:,j])
    #     jac.append(jac_i)
    # jac = np.concatenate(jac, 0)

    # jac = compute_jacobian_of_residual_wrt_T(Pi, X).reshape(-1)

    jac_T1 = compute_jacobian_of_residual_wrt_T1(Pi, X).T.reshape(-1)
    jac_T2 = compute_jacobian_of_residual_wrt_T2(Pi, X).T.reshape(-1)
    jac_T3 = compute_jacobian_of_residual_wrt_T3(Pi, X).T.reshape(-1)

    return res, jac_T1, jac_T2, jac_T3

def compute_update(res, jac, mu):
    I = np.eye(jac.shape[1])
    delta = -LA.inv(jac.T @ jac + mu*I) @ (jac.T @ res)
    return delta

def optimize_T(P_arr, X_init, X_idx_arr, x_arr, inliers_arr, mu_init, n_its, verbose=False):

    if verbose:
        print('\nReprojection error before bundle adjustment:')
        _, _ = compute_total_reprojection_error_wrt_T(P_arr, X_init, X_idx_arr, x_arr, inliers_arr, verbose=True)

    n_cameras = P_arr.shape[0]
    steps = []

    for i in trange(n_cameras):

        Pi = P_arr[i]
        Ri = Pi[:,:-1]
        Ti = Pi[:,-1]

        if not np.isnan(Ti[0]):
            
            X_idx = X_idx_arr[i]
            inliers = inliers_arr[i]

            X = X_init[:,X_idx][:,inliers]
            x = x_arr[i][:,inliers]

            converged = False
            mu = mu_init
            step = 0
        
            while (step <= n_its) and converged is not True:
                step += 1
                
                res, jac_T1, jac_T2, jac_T3 = linearize_reprojection_error_wrt_T(Pi, X, x)

                delta_T1 = compute_update(res[:,None], jac_T1[:,None], mu)
                delta_T2 = compute_update(res[:,None], jac_T2[:,None], mu)
                delta_T3 = compute_update(res[:,None], jac_T3[:,None], mu)

                Ti_opt = np.array([Ti[0] + delta_T1[0,0], 
                                   Ti[1] + delta_T2[0,0], 
                                   Ti[2] + delta_T3[0,0]])

                Pi_opt = np.column_stack((Ri, Ti_opt[:,None]))
                reproj_err, _ = compute_reprojection_error_wrt_T(Pi, X, x)
                reproj_err_opt, _ = compute_reprojection_error_wrt_T(Pi_opt, X, x)

                if np.isclose(reproj_err_opt, reproj_err):
                    converged = False
                elif reproj_err_opt < reproj_err:
                    Ti = Ti_opt
                    mu /= 10
                else:
                    mu *= 10
                # print(step, end='\r')

            P_arr[i,:,-1] = Ti.copy()
            steps.append(step)

    if verbose:
        print('\nReprojection error after bundle adjustment:')
        _, _ = compute_total_reprojection_error_wrt_T(P_arr, X_init, X_idx_arr, x_arr, inliers_arr, verbose=True)

        print('\nAvg its:', np.mean(steps))
        print('Max its:', np.max(steps))
        print('Min its:', np.min(steps))

    return P_arr

In [203]:
abs_rots = np.load('data/dataset_{}_RA_abs_rots.npy'.format(data_set))
trans = np.load('data/dataset_{}_TR_trans.npy'.format(data_set))

RA = False
opt = True

if RA:
    trans = np.load('data/dataset_{}_RA_trans.npy'.format(data_set))

if opt:
    trans = np.load('data/dataset_{}_LM_trans_opt.npy'.format(data_set))

cameras = []
X_idx_arr = []
x_arr = []
inliers_arr = []

for i in range(trans.shape[0]):
    R = abs_rots[i]
    T = trans[i]
    cameras.append(np.column_stack((R, T)))
    
    X_idx_arr.append(np.load('data/dataset_{}_TR_X_idx_{}.npy'.format(data_set, i)))
    x_norm = np.load('data/dataset_{}_TR_x2_norm_{}.npy'.format(data_set, i))
    x_arr.append(x_norm)
    if RA:
        inliers_T = np.ones(x_norm.shape[1], dtype=bool)
    else:
        inliers_T = np.load('data/dataset_{}_TR_T_inliers_{}.npy'.format(data_set, i))
    inliers_arr.append(inliers_T)
    print(cameras[i])

cameras = np.array(cameras)
print('No. cameras:', cameras.shape[0])

[[ 1.00000000e+00  0.00000000e+00  0.00000000e+00 -1.39368273e-05]
 [ 0.00000000e+00  1.00000000e+00  0.00000000e+00  1.36220187e-04]
 [ 0.00000000e+00  0.00000000e+00  1.00000000e+00  7.63322784e-05]]
[[ 0.99719066  0.00850512  0.07442073 -0.09706431]
 [-0.01103445  0.99937303  0.03364202 -0.00115319]
 [-0.07408794 -0.0343687   0.9966593  -0.03011289]]
[[ 0.98932379  0.03546876  0.14135203 -0.23270646]
 [-0.03560604  0.99936469 -0.00155872  0.00658809]
 [-0.14131751 -0.0034909   0.98995817 -0.04500063]]
[[ 0.97923546  0.05363245  0.19550312 -0.3542573 ]
 [-0.04969116  0.99845138 -0.02501266  0.00556536]
 [-0.19654185  0.01477851  0.98038406 -0.07285779]]
[[ 0.9674886   0.05787766  0.24620314 -0.46633508]
 [-0.054916    0.99831238 -0.01888432  0.00538504]
 [-0.24688062  0.00474988  0.96903426 -0.07621733]]
[[ 0.94580839  0.09362227  0.31093626 -0.59726071]
 [-0.09048362  0.9955956  -0.02453798  0.02289115]
 [-0.31186408 -0.00492641  0.95011396 -0.05182552]]
[[ 0.92721887  0.09872628  0

In [185]:
mu_init = 1
n_its = 1000
cameras_opt = optimize_T(cameras.copy(), X_init_feasible_inliers, X_idx_arr, x_arr, inliers_arr, mu_init, n_its, verbose=True)


Reprojection error before bundle adjustment:

Total reprojection error: 62486.97
Median reprojection error: 8928.86
Avg. reprojection error: 6943.0


100%|██████████| 9/9 [00:28<00:00,  3.18s/it]


Reprojection error after bundle adjustment:

Total reprojection error: 62360.65
Median reprojection error: 8899.58
Avg. reprojection error: 6928.96

Avg its: 1001.0
Max its: 1001
Min its: 1001


In [217]:
print('\n\n\n### Triangulating final 3D-reconstruction ###\n')

X_final = []
reconstruction = True
opt_eriks_LM = False

if opt_eriks_LM:
    cameras = cameras_opt.copy()

valid_idx = []
for i in range(keep_rots.shape[0]):
    if keep_rots[i]:
        valid_idx.append(i)
valid_idx = np.array(valid_idx)
print(valid_idx)

if reconstruction:
    for idx in range(valid_idx.shape[0]-1):
        i = valid_idx[idx]
        P1 = cameras[i]
        P2 = cameras[valid_idx[idx+1]]        

        x1 = np.load('data/dataset_{}_RA_x1_{}.npy'.format(data_set, i))
        x2 = np.load('data/dataset_{}_RA_x2_{}.npy'.format(data_set, i))
        x1_norm = cv.dehomogenize(K_inv @ x1)
        x2_norm = cv.dehomogenize(K_inv @ x2)
        
        inliers = np.load('data/dataset_{}_RA_E_inliers_{}.npy'.format(data_set, i))
        x1_norm_inliers = x1_norm[:,inliers]
        x2_norm_inliers = x2_norm[:,inliers]

        percentile = 90
        X_inliers = cv.triangulate_3D_point_DLT(P1, P2, x1_norm_inliers, x2_norm_inliers, verbose=False)
        feasible_pts = compute_feasible_points(P1, P2, X_inliers, percentile)
        X_final.append(X_inliers[:,feasible_pts])

    X_final = np.column_stack(X_final)
else:
    X_final = [0,0,0]

C_arr, axis_arr = cv.compute_camera_center_and_normalized_principal_axis(cameras[valid_idx], multi=True)
plot_cameras_and_3D_points(X_final, C_arr, axis_arr, s=0.5, valid_idx=valid_idx, origin=init_pair[0], path=None, save=False)




### Triangulating final 3D-reconstruction ###

[0 1 2 3 4 5 6]
